In [20]:
import crocoddyl
import casadi as cs
import numpy as np

class ActionModelCasadi(crocoddyl.ActionModelAbstract):
    """
    A custom Crocoddyl action model that uses CasADi for automatic differentiation.
    """

    def __init__(self, state, nu, cost_weights, dt, m_cart, m_pole, l_pole, grav):
        """
        Initialize the CasADi action model.

        Args:
            state (crocoddyl.StateAbstract): The state representation.
            nu (int): The dimension of the control input.
            cost_weights (list): A list of weights for the cost function.
            dt (float): The time step for integration.
            m_cart (float): The mass of the cart.
            m_pole (float): The mass of the pole.
            l_pole (float): The length of the pole.
            grav (float): The gravitational acceleration.
        """
        crocoddyl.ActionModelAbstract.__init__(self, state, nu)

        # System parameters
        self.Δt = dt
        self.m_cart = m_cart
        self.m_pole = m_pole
        self.l_pole = l_pole
        self.grav = grav
        self.costWeights = cost_weights

        # Define CasADi symbolic variables
        x_cas = cs.SX.sym("x", self.state.nx)
        u_cas = cs.SX.sym("u", self.nu)

        y, θ, ẏ, θ̇ = x_cas[0], x_cas[1], x_cas[2], x_cas[3]
        f = u_cas[0]

        # Shortname for system parameters
        sin_θ, cos_θ = cs.sin(θ), cs.cos(θ)

        # Cartpole dynamics
        μ = self.m_cart + self.m_pole * sin_θ * sin_θ
        ÿ = f / μ + self.m_pole * self.grav * cos_θ * sin_θ / μ - self.m_pole * self.l_pole * (sin_θ * θ̇ ** 2 / μ)
        θ̈ = (f / self.l_pole) * cos_θ / μ + ((self.m_cart + self.m_pole) * self.grav / self.l_pole) * sin_θ / μ - self.m_pole * cos_θ * sin_θ * θ̇**2 / μ

        # Forward Euler integration
        ẏ_next = ẏ + self.Δt * ÿ
        θ̇_next = θ̇ + self.Δt * θ̈
        y_next = y + self.Δt * ẏ_next
        θ_next = θ + self.Δt * θ̇_next

        xnext = cs.vertcat(y_next, θ_next, ẏ_next, θ̇_next)

        # Cost function
        cost_r = self.costWeights * cs.vertcat(sin_θ, 1.0 - cos_θ, y, ẏ, θ̇, f)
        cost = 0.5 * cs.sumsqr(cost_r)

        # Create CasADi functions
        self.dynamics_func = cs.Function('dyn', [x_cas, u_cas], [xnext])
        self.cost_func = cs.Function('cost', [x_cas, u_cas], [cost])
        self.cost_r_func = cs.Function('cost_r', [x_cas, u_cas], [cost_r])

        # Derivatives
        self.Fx = cs.Function('Fx', [x_cas, u_cas], [cs.jacobian(xnext, x_cas)])
        self.Fu = cs.Function('Fu', [x_cas, u_cas], [cs.jacobian(xnext, u_cas)])
        self.Lx = cs.Function('Lx', [x_cas, u_cas], [cs.jacobian(cost, x_cas)])
        self.Lu = cs.Function('Lu', [x_cas, u_cas], [cs.jacobian(cost, u_cas)])
        self.Lxx = cs.Function('Lxx', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, x_cas), x_cas)])
        self.Luu = cs.Function('Luu', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, u_cas), u_cas)])
        self.Lxu = cs.Function('Lxu', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, x_cas), u_cas)])


    def calc(self, data, x, u=None):
        """
        Computes the next state and the cost function.
        """
        if u is None:
            u = np.zeros(self.nu)

        data.xnext = np.asarray(self.dynamics_func(x, u)).flatten()
        data.cost = np.asarray(self.cost_func(x, u)).item()
        data.r = np.asarray(self.cost_r_func(x,u)).flatten()

        return data.xnext, data.cost

    def calcDiff(self, data, x, u=None):
        """
        Computes the derivatives of the dynamics and the cost function.
        """
        if u is None:
            u = np.zeros(self.nu)

        # Compute derivatives using CasADi functions
        data.Fx = np.asarray(self.Fx(x, u))
        data.Fu = np.asarray(self.Fu(x, u))
        data.Lx = np.asarray(self.Lx(x, u)).flatten()
        data.Lu = np.asarray(self.Lu(x, u)).flatten()
        data.Lxx = np.asarray(self.Lxx(x, u))
        data.Luu = np.asarray(self.Luu(x, u))
        data.Lxu = np.asarray(self.Lxu(x, u))


if __name__ == '__main__':
    # Define state and action model
    state = crocoddyl.StateVector(4)
    nu = 1
    
    # System parameters
    dt = 0.01
    m_cart = 1.0
    m_pole = 0.1
    l_pole = 0.5
    grav = 9.81
    
    # Cost weights
    cost_weights = np.array([1.0, 1.0, 0.1, 0.1, 0.1, 0.001])
    
    actuation = ActionModelCasadi(state, nu, cost_weights, dt, m_cart, m_pole, l_pole, grav)

    # Create data structures
    action_data = actuation.createData()
    
    # Define an initial state and control
    x0 = np.array([0., np.pi, 0., 0.])
    u0 = np.array([0.])
    
    # Compute and display the dynamics and cost
    actuation.calc(action_data, x0, u0)
    print("Next state:", action_data.xnext)
    print("Cost:", action_data.cost)
    
    # Compute and display the derivatives
    actuation.calcDiff(action_data, x0, u0)
    print("Fx:\n", action_data.Fx)
    print("Fu:\n", action_data.Fu)
    print("Lx:", action_data.Lx)
    print("Lu:", action_data.Lu)

Next state: [-1.20137851e-20  3.14159265e+00 -1.20137851e-18  2.64303272e-17]
Cost: 2.0
Fx:
 [[ 1.000000e+00  9.810000e-05  1.000000e-02 -0.000000e+00]
 [ 0.000000e+00  9.978418e-01  0.000000e+00  1.000000e-02]
 [ 0.000000e+00  9.810000e-03  1.000000e+00 -0.000000e+00]
 [ 0.000000e+00 -2.158200e-01  0.000000e+00  1.000000e+00]]
Fu:
 [ 0.0001 -0.0002  0.01   -0.02  ]
Lx: [0.0000000e+00 1.2246468e-16 0.0000000e+00 0.0000000e+00]
Lu: [0.]


In [19]:
# ... (the previous __main__ block)

# Define the OCP
x0 = np.array([0., 0, 0., 0.]) 
T = 100 # Number of knots

# Create a list of action models for the OCP
running_models = [actuation] * T
terminal_model = ActionModelCasadi(state, nu, cost_weights * 10, dt, m_cart, m_pole, l_pole, grav) # Higher terminal cost

problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)

# Create a solver
solver = crocoddyl.SolverDDP(problem)
solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

# Solve the OCP
xs = [x0] * (T + 1)
us = [np.zeros(nu)] * T
solver
solver.solve(xs, us)


iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   1  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00


True

In [7]:
    # Define state and action model
    state = crocoddyl.StateVector(4)
    nu = 1
    
    # System parameters
    dt = 0.01
    m_cart = 1.0
    m_pole = 0.1
    l_pole = 0.5
    grav = 9.81
    
    # Cost weights
    cost_weights = np.array([1.0, 1.0, 0.1, 0.1, 0.1, 0.001])
    
    actuation = ActionModelCasadi(state, nu, cost_weights, dt, m_cart, m_pole, l_pole, grav)

    # Create data structures
    action_data = actuation.createData()
    
    # Define an initial state and control
    x0 = np.array([0., np.pi, 0., 0.])
    u0 = np.array([0.])
    
    # Compute and display the dynamics and cost
    actuation.calc(action_data, x0, u0)
    print("Next state:", action_data.xnext)
    print("Cost:", action_data.cost)
    
    # Compute and display the derivatives
    actuation.calcDiff(action_data, x0, u0)
    print("Fx:\n", action_data.Fx)
    print("Fu:\n", action_data.Fu)
    print("Lx:", action_data.Lx)
    print("Lu:", action_data.Lu)

Next state: [-1.20137851e-20  3.14159265e+00 -1.20137851e-18  2.64303272e-17]
Cost: 2.0
Fx:
 [[ 1.000000e+00  9.810000e-05  1.000000e-02 -0.000000e+00]
 [ 0.000000e+00  9.978418e-01  0.000000e+00  1.000000e-02]
 [ 0.000000e+00  9.810000e-03  1.000000e+00 -0.000000e+00]
 [ 0.000000e+00 -2.158200e-01  0.000000e+00  1.000000e+00]]
Fu:
 [ 0.0001 -0.0002  0.01   -0.02  ]
Lx: [0.0000000e+00 1.2246468e-16 0.0000000e+00 0.0000000e+00]
Lu: [0.]


In [3]:
# --- 1. Define Model and Problem ---

# State and control dimensions
state = crocoddyl.StateVector(4)
nu = 1

# Cost weights
# Weights for [y, θ, ẏ, θ̇, f]
running_weights = np.array([1., 1., 0.1, 0.1, 0.001])
terminal_weights = np.array([100., 100., 10., 10., 0.]) # Higher terminal cost, no control cost

# Create running and terminal action models
model = ActionModelCasadi(state, nu, running_weights)
model_term = ActionModelCasadi(state, nu, terminal_weights, is_terminal=True)

# Define the Optimal Control Problem
T = 100  # Number of knots (time steps) in the trajectory
x0 = np.array([0.0, 3.14, 1.0, 0.5]) # Initial state [y, θ, ẏ, θ̇] (starting upside down)

# The shooting problem is defined by the initial state, a list of running models,
# and a terminal model.
problem = crocoddyl.ShootingProblem(x0, [model] * T, model_term)

# --- 2. Create Solver and Solve ---

# Use the Differential Dynamic Programming (DDP) solver
ddp = crocoddyl.SolverDDP(problem)

# Add callbacks for logging and printing solver progress
ddp.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

print("--- Solving OCP with DDP ---")
# Solve the problem with default initial guesses (zero controls)
conv = ddp.solve()

# --- 3. Print Results ---
print("\n--- Solver Results ---")
if conv:
    print("Convergence: ACHIEVED")
    print(f"Final cost: {ddp.cost}")
else:
    print("Convergence: FAILED")

print(f"Final state: {ddp.xs[-1]}")

TypeError: ActionModelCasadi.__init__() missing 5 required positional arguments: 'dt', 'm_cart', 'm_pole', 'l_pole', and 'grav'

In [8]:
import crocoddyl
import casadi as cs
import numpy as np

class ActionModelCasadi(crocoddyl.ActionModelAbstract):
    """
    A custom Crocoddyl action model that uses CasADi for automatic differentiation
    of the cart-pole system.
    """

    def __init__(self, state, nu, cost_weights, is_terminal=False):
        """
        Initialize the CasADi action model for the cart-pole.

        Args:
            state (crocoddyl.StateAbstract): The state representation.
            nu (int): The dimension of the control input.
            cost_weights (np.ndarray): An array of weights for the cost function.
            is_terminal (bool): Flag to indicate if this is a terminal model.
        """
        crocoddyl.ActionModelAbstract.__init__(self, state, nu)

        # System parameters
        self.Δt = 0.01
        self.m_cart = 1.0
        self.m_pole = 0.1
        self.l_pole = 0.5
        self.grav = 9.81
        self.costWeights = cost_weights
        self.is_terminal = is_terminal

        # Define CasADi symbolic variables for state and control
        x_cas = cs.SX.sym("x", self.state.nx)
        u_cas = cs.SX.sym("u", self.nu)

        # Unpack state variables
        y, θ, ẏ, θ̇ = x_cas[0], x_cas[1], x_cas[2], x_cas[3]
        
        # Handle control variable, which is zero for the terminal node
        f = u_cas[0] if not self.is_terminal else 0.0

        # Dynamics of the cart-pole system
        sin_θ, cos_θ = cs.sin(θ), cs.cos(θ)
        μ = self.m_cart + self.m_pole * sin_θ * sin_θ
        
        ÿ_num = f + self.m_pole * sin_θ * (self.l_pole * θ̇**2 + self.grav * cos_θ)
        ÿ = ÿ_num / μ
        
        θ̈_num = -f * cos_θ - self.m_pole * self.l_pole * θ̇**2 * cos_θ * sin_θ - \
                (self.m_cart + self.m_pole) * self.grav * sin_θ
        θ̈ = θ̈_num / (self.l_pole * μ)

        # Forward Euler integration to get the next state
        ẏ_next = ẏ + self.Δt * ÿ
        θ̇_next = θ̇ + self.Δt * θ̈
        y_next = y + self.Δt * ẏ_next
        θ_next = θ + self.Δt * θ̇_next

        xnext = cs.vertcat(y_next, θ_next, ẏ_next, θ̇_next)

        # Cost function (residuals-based)
        # The goal is to bring the system to an upright and stationary state (y=0, θ=0, ẏ=0, θ̇=0)
        # with minimal control effort.
        # Note: we use (θ) directly as a residual, assuming the goal is θ=0.
        cost_residuals = cs.vertcat(y, θ, ẏ, θ̇, f)
        cost = 0.5 * cs.sumsqr(cs.diag(self.costWeights) @ cost_residuals)

        # Create CasADi functions for dynamics and cost
        self.dynamics_func = cs.Function('dyn', [x_cas, u_cas], [xnext])
        self.cost_func = cs.Function('cost', [x_cas, u_cas], [cost])
        
        # Automatic differentiation to get derivatives
        # Jacobians
        self.Fx = cs.Function('Fx', [x_cas, u_cas], [cs.jacobian(xnext, x_cas)])
        self.Fu = cs.Function('Fu', [x_cas, u_cas], [cs.jacobian(xnext, u_cas)])
        self.Lx = cs.Function('Lx', [x_cas, u_cas], [cs.jacobian(cost, x_cas)])
        self.Lu = cs.Function('Lu', [x_cas, u_cas], [cs.jacobian(cost, u_cas)])
        # Hessians
        self.Lxx = cs.Function('Lxx', [x_cas, u_cas], [cs.hessian(cost, x_cas)[0]])
        self.Luu = cs.Function('Luu', [x_cas, u_cas], [cs.hessian(cost, u_cas)[0]])
        self.Lxu = cs.Function('Lxu', [x_cas, u_cas], [cs.jacobian(cs.gradient(cost, u_cas), x_cas)])

    def calc(self, data, x, u=None):
        """ Computes the next state and cost """
        if self.is_terminal:
            # For the terminal node, control is not defined
            u = np.zeros(self.nu)
        
        data.xnext = np.asarray(self.dynamics_func(x, u)).flatten()
        data.cost = np.asarray(self.cost_func(x, u)).item()
        
        return data.xnext, data.cost

    def calcDiff(self, data, x, u=None):
        """ Computes the derivatives of the dynamics and cost """
        if self.is_terminal:
            u = np.zeros(self.nu)

        # Compute and store derivatives using pre-compiled CasADi functions
        data.Fx = np.asarray(self.Fx(x, u))
        data.Fu = np.asarray(self.Fu(x, u))
        data.Lx = np.asarray(self.Lx(x, u)).flatten()
        data.Lu = np.asarray(self.Lu(x, u)).flatten()
        data.Lxx = np.asarray(self.Lxx(x, u))
        data.Luu = np.asarray(self.Luu(x, u))
        data.Lxu = np.asarray(self.Lxu(x, u))

if __name__ == '__main__':
    # --- 1. Define Model and Problem ---
    
    # State and control dimensions
    state = crocoddyl.StateVector(4)
    nu = 1

    # Cost weights
    # Weights for [y, θ, ẏ, θ̇, f]
    running_weights = np.array([1., 1., 0.1, 0.1, 0.001])
    terminal_weights = np.array([100., 100., 10., 10., 0.]) # Higher terminal cost, no control cost

    # Create running and terminal action models
    model = ActionModelCasadi(state, nu, running_weights)
    model_term = ActionModelCasadi(state, nu, terminal_weights, is_terminal=True)

    # Define the Optimal Control Problem
    T = 100  # Number of knots (time steps) in the trajectory
    x0 = np.array([0.0, 3.14, 1.0, 0.5]) # Initial state [y, θ, ẏ, θ̇] (starting upside down)

    # The shooting problem is defined by the initial state, a list of running models,
    # and a terminal model.
    problem = crocoddyl.ShootingProblem(x0, [model] * T, model_term)

    # --- 2. Create Solver and Solve ---
    
    # Use the Differential Dynamic Programming (DDP) solver
    ddp = crocoddyl.SolverDDP(problem)

    # Add callbacks for logging and printing solver progress
    ddp.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

    print("--- Solving OCP with DDP ---")
    # Solve the problem with default initial guesses (zero controls)
    conv = ddp.solve()

    # # --- 3. Print Results ---
    # print("\n--- Solver Results ---")
    # if conv:
    #     print("Convergence: ACHIEVED")
    #     print(f"Final cost: {ddp.cost}")
    # else:
    #     print("Convergence: FAILED")

    # print(f"Final state: {ddp.xs[-1]}")
    
    # # To access the solution:
    # # ddp.xs contains the optimal state trajectory
    # # ddp.us contains the optimal control sequence

--- Solving OCP with DDP ---
